##  hello，装饰器

**装饰器**放在一个函数开始定义的地方，它就像一顶帽子一样戴在这个函数的头上。和这个函数绑定在一起。在我们调用这个函数的时候，第一件事并不是执行这个函数，而是将这个函数做为参数传入它头顶上这顶帽子，这顶帽子我们称之为 装饰器 。

你要问我装饰器可以实现什么功能？我只能说你的脑洞有多大，装饰器就有多强大。

装饰器的使用方法很固定：

+ 先定义一个装饰函数（帽子）（也可以用类、偏函数实现）

+ 再定义你的业务函数、或者类（人）

+ 最后把这顶帽子带在这个人头上

装饰器的简单的用法有很多，这里举两个常见的:

1. 日志打印器
2. 时间计时器


## 入门用法：日志打印器

首先是日志打印器。实现的功能：

> 在函数执行前，先打印一行日志告知一下主人，我要执行函数了。在函数执行完，也不能拍拍屁股就走人了，再打印一行日志告诉主人，我执行完了。

In [1]:
# 这是装饰函数
def logger(func):
    def wrapper(*args, **kw):
        print('我要准备开始计算： {}函数了'.format(func.__name__))
        
        # 真正要执行的函数
        func(*args, **kw)
        
        print('啊哈， 我计算完了。给自己加个鸡腿🍗！！')
        
    return wrapper

In [2]:
@logger
def add(x,y):
    print('{} + {} = {}'.format(x,y,x+y))

In [3]:
add(3,5)

我要准备开始计算： add函数了
3 + 5 = 8
啊哈， 我计算完了。给自己加个鸡腿🍗！！


## 入门用法：时间计时器

在来看看时间计时器实现的功能：

> 顾名思义就是计算一个函数运行所需要的时间。

In [4]:
import time

# 这是装饰函数，即装饰器
def timer(func):
    def wrapper(*args, **kw):
        t1 = time.time()
        
        func(*args, **kw)
        
        t2 = time.time()
        # 计算一下所需的时间
        cost_time = t2 - t1
        print("花费的时间是：{}秒".format(cost_time))
    return wrapper

In [5]:
# 假如我们让函数睡眠10秒～

@timer
def sleep_second(sleep_time):
    
    time.sleep(sleep_time)

In [6]:
# 运行一下看看👀
sleep_second(10)

花费的时间是：10.005182981491089秒


## 进阶用法：带参数的函数装饰器

上面的例子，装饰器是不能接收参数的。其用法，只能适用于一些简单的场景。不传参的装饰器，只能对被装饰函数，执行固定逻辑。

如果你有经验，你一定经常在项目中，看到有的装饰器是带有参数的。

装饰器本身是一个函数，既然做为一个函数都不能携带函数，那这个函数的功能就很受限。只能执行固定的逻辑。这无疑是非常不合理的。而如果我们要用到两个内容大体一致，只是某些地方不同的逻辑。不传参的话，我们就要写两个装饰器。

那么装饰器如何实现传参呢，会比较复杂，需要两层嵌套。

In [7]:
def say_hello(country):
    def wrapper(func):
        def deco(*args, **kw):
            if country == 'china':
                print('你好！')
            if country == 'america':
                print('hello！')
            # 真正执行函数的地方
            func(*args, **kw)
        return deco
    return wrapper

In [8]:
@say_hello('china')
def chinese():
    print('我来自中国。')

In [9]:
chinese()

你好！
我来自中国。


In [10]:
@say_hello('america')
def american():
    print("I'm from American")

In [11]:
american()

hello！
I'm from American


## 高阶用法：不带参数的类装饰器

以上都是基于函数实现的装饰器，在阅读别人代码是，还可以时常发现还有基于类实现的装饰器。

基于类装饰器的实现，必须实现`__call__`和`__init__`两个内置函数。其中：  

`__init__`:  接收被装饰函数

`__call__`:  实现装饰逻辑

In [12]:
class logger(object):
    def __init__(self, func):
        self._func = func
        
    def __call__(self, *args, **kwargs):
        print("[INFO]: the function {func}() is running...".format(func=self._func.__name__))
        
        return self._func(*args, **kwargs)

In [13]:
@logger
def say(something):
    print('say {}!'.format(something))

In [14]:
say('你好')

[INFO]: the function say() is running...
say 你好!


## 高阶用法：带参数的类装饰器

上面不带参数的例子，只能打印INFO级别的日志。正常情况下，我们还需要打印DEBUG, WARNING等级别的日志。这就需要给类装饰器传入参数，给这个函数指定级别了。

带参数和不带参数的类装饰器有很大的不同。

`__init__`: 不在接收被装饰函数，而是接收传入参数。  

`__call__`: 接收被装饰函数，实现装饰逻辑。

In [15]:
class logger(object):
    def __init__(self, level="INFO"):
        self.level = level
        
    def __call__(self, func):  # 接收被装饰函数
        def wrapper(*args, **kwargs):
            print("[{level}]: the function {func}() is running...".format(level=self.level, func=func.__name__))
            func(*args, **kwargs)
        return wrapper  # 返回函数

In [16]:
@logger(level='WARNING')
def say(something):
    print('say: {}'.format(something))

In [17]:
say('你好')

[WARNING]: the function say() is running...
say: 你好


## wraps装饰器有啥用？

在functools标准库中有提供一个wraps装饰器，你应该也经常看到过，那它有什么作用呢？

In [18]:
def wrapper(func):
    def inner_function():
        pass
    
    return inner_function

@wrapper
def wrapped():
    pass

print(wrapped.__name__) # inner_function

inner_function


为什么会是这样？不是应该返回func吗？

这也不难理解，因为上边执行func和下边的decorator(func)是等价的，所以上面func.__name__是等价于下面的decorator(func).__name__的，那当然名字是inner_function.

In [19]:
def wrapper(func):
    def inner_function():
        pass
    
    return inner_function

@wrapper
def wrapped():
    pass

print(wrapper(wrapped).__name__) # inner_function

inner_function


那如何避免这种情况的产生？方法是使用 functools .wraps 装饰器，它的作用就是将 被修饰的函数(wrapped) 的一些属性值赋值给 修饰器函数(wrapper) ，最终让属性的显示更符合我们的直觉。

In [20]:
from functools import wraps

def wrapper(func):
    @wraps(func)
    def inner_function():
        pass
    
    return inner_function
    
    
@wrapper
def wrapped():
    pass

print(wrapped.__name__) # wrapped

wrapped


## 使用偏函数与类实现装饰器

绝大多数装饰器都是基于函数和闭包实现的，但这并非制造装饰器的唯一方式。

事实上，Python 对某个对象是否能通过装饰器（ @decorator）形式使用只有一个要求：decorator 必须是一个“可被调用（callable）的对象。

对于这个 callable 对象，我们最熟悉的就是函数了。

除函数之外，类也可以是 callable 对象，只要实现了`__call__` 函数（上面几个盒子已经接触过了），还有比较少人使用的偏函数也是 callable 对象。

接下来就来说说，如何使用 类和偏函数结合实现一个与众不同的装饰器。

In [21]:
import time 
import functools

class DelayFunc:
    
    def __init__(self, duration, func):
        self.duration = duration
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print(f"Wait for {self.duration} seconds...")
        time.sleep(self.duration)
        return self.func(*args, **kwargs)
    
    def eager_call(self, *args, **kwargs):
        print('call without delay')
        return self.func(*args, **kwargs)
    
    
def delay(duration):
    """
    装饰器： 推迟某个函数的执行
    同时，提供 .eager_call方法立即执行
    """
    return functools.partial(DelayFunc, duration)

In [22]:
@delay(duration=2)
def add(a,b):
    return a+b

In [23]:
add # 可见，add变成了Delay的实例

In [24]:
add(3,5) # 直接调用实例， 进入__call__

Wait for 2 seconds...


8

In [25]:
add.func # 实现实例方法

<function __main__.add>

## 内置装饰器：property

以上，我们介绍的都是自定义的装饰器。

其实Python语言本身也有一些装饰器。比如property这个内建装饰器，我们再熟悉不过了。

它通常存在于类中，可以将一个函数定义成一个属性，属性的值就是该函数return的内容。

通常我们给实例绑定属性是这样的

In [26]:
class Student(object):
    def __init__(self, name, age=None):
        self.name = name
        self.age = age
        

In [27]:
# 实例化
XiaoMing = Student('小明')

In [28]:
# 添加属性
XiaoMing.age = 25

In [29]:
# 查询属性
XiaoMing.age

25

In [30]:
# 删除属性
del XiaoMing.age

但是稍有经验的开发人员，一下就可以看出，这样直接把属性暴露出去，虽然写起来很简单，但是并不能对属性的值做合法性限制。为了实现这个功能，我们可以这样写。

In [31]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
    def set_age(self, age):
        if not isinstance(age, int):
            raise ValueError('输入不合法：年龄必须为数值！')
        if not 0< age < 100:
            raise ValueError("输入不合法：年龄范围必须为0～100")
            
        self._age = age
        
    def get_age(self):
        return self._age
    
    def del_age(self):
        self._age = None

In [32]:
XiaoMing = Student("小明")

In [33]:
# 添加属性
XiaoMing.set_age(25)

In [34]:
# 查询属性
XiaoMing.get_age()

25

In [35]:
# 删除属性
XiaoMing.del_age()

上面的代码设计虽然可以变量的定义，但是可以发现不管是获取还是赋值（通过函数）都和我们平时见到的不一样。 按照我们思维习惯应该是这样的。

```
# 赋值
XiaoMing.age = 25

# 获取
XiaoMing.age
```

那么，这样的获取方式如何实现呢？

In [36]:
class Student(object):
    def __init__(self,name):
        self.name = name
        self._age = None
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise ValueError("输入不合法：年龄必须为数值")
        if not 0 < value < 100:
            raise ValueError("输入不合法：年龄必须为0～100")
            
        self._age = value
        
    @age.deleter
    def age(self):
        del self._age

In [37]:
XiaoMing = Student('小明')

In [38]:
XiaoMing.age = 25

In [39]:
XiaoMing.age

25

In [40]:
del XiaoMing.age

用**`@property`**装饰过的函数，会将一个函数定义成一个属性，属性的值就是该函数return的内容。同时，会将这个函数变成另外一个装饰器。就像后面我们使用的**`@age.setter`**和**`@age.deleter`**。

**`@age.setter`** 使得我们可以使用XiaoMing.age = 25这样的方式直接赋值。

**`@age.deleter`**使得我们可以使用del XiaoMing.age这样的方式来删除属性。